<a href="https://colab.research.google.com/github/will-o-thewisp/Wordplays/blob/first-steps/Wordplays%2C_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# intro import
import pandas as pd
from google.colab import drive
import re
import json

from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')

!pip install pymorphy2 > None
import pymorphy2
drive.mount('drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


# Data preprocessing

In [ ]:
# open json file with messages
with open('/content/drive/MyDrive/Pet projects/Wordplays/Ekaterina Schulmans channel posts.json') as user_file:
  file_contents = user_file.read()

parsed_json = json.loads(file_contents)
tgposts = pd.json_normalize(parsed_json['messages'])
tgposts.shape

(6169, 33)

In [ ]:
# making texts... texts

def extract_text(text):
    if type(text) is str:
      return text

    extracted_text = ""
    for part in text:
        if type(part) is dict:
             extracted_text = extracted_text + part['text']
        else:
            extracted_text = extracted_text + part
    return extracted_text

tgposts['text'] = tgposts['text'].apply(extract_text)
tgposts.head()

,id,type,date,date_unixtime,actor,actor_id,action,title,text,text_entities,...,edited_unixtime,thumbnail,performer,author,sticker_emoji,message_id,poll.question,poll.closed,poll.total_voters,poll.answers
0,1,service,2018-12-13T13:57:02,1544698622,Екатерина Шульман,channel1485724714,create_channel,Shulman Ekaterina,,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,service,2018-12-13T14:00:10,1544698810,Екатерина Шульман,channel1485724714,edit_group_title,Екатерина Шульман,,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,message,2018-12-20T08:30:15,1545283815,NaN,NaN,NaN,NaN,Два условия обеспечат вам победу во взаимодейс...,"[{'type': 'plain', 'text': 'Два условия обеспе...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,message,2018-12-21T05:45:19,1545360319,NaN,NaN,NaN,NaN,"Павел Чиков, конечно, человек-зверь, прям Росо...","[{'type': 'text_link', 'text': 'Павел Чиков', ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,message,2018-12-22T10:00:15,1545462015,NaN,NaN,NaN,NaN,Ссылка на Новую газету об этом деле с подробно...,"[{'type': 'text_link', 'text': 'Ссылка', 'href...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# cutting off the columns I don't need
tgposts = tgposts[['type', 'date', 'text']]
tgposts.head()

,type,date,text
0,service,2018-12-13T13:57:02,
1,service,2018-12-13T14:00:10,
2,message,2018-12-20T08:30:15,Два условия обеспечат вам победу во взаимодейс...
3,message,2018-12-21T05:45:19,"Павел Чиков, конечно, человек-зверь, прям Росо..."
4,message,2018-12-22T10:00:15,Ссылка на Новую газету об этом деле с подробно...


In [ ]:
# getting rid of unnecessary service messages, e.g. 'Channel photo updated' etc.
tgposts = tgposts.loc[tgposts['type'] == 'message']
tgposts = tgposts[['date', 'text']]
tgposts.head()

,date,text
2,2018-12-20T08:30:15,Два условия обеспечат вам победу во взаимодейс...
3,2018-12-21T05:45:19,"Павел Чиков, конечно, человек-зверь, прям Росо..."
4,2018-12-22T10:00:15,Ссылка на Новую газету об этом деле с подробно...
5,2018-12-23T15:01:59,https://www.youtube.com/watch?v=nLGozExwvio
6,2018-12-24T09:00:26,​​Часто спрашивают о чем почитать про историю ...


In [ ]:
# adding a new column to move preprocessed lemmatized texts there
tgposts['preprocessed'] = ""
tgposts.shape

(6151, 3)

In [ ]:
# creating function to remove so called foreign agent disclaimer

def remove_yebala(text):
  justtext = re.sub(r'[А-Я, ()]{310,333}', "", text)
  return justtext

In [ ]:
# removing so called foreign agent disclaimer
tgposts['text'] = tgposts['text'].apply(remove_yebala)
tgposts.tail()

,date,text,preprocessed
6164,2024-01-11T19:55:27,"Cсылочка для тех, кто в России без VPN:\n\nhtt...",
6165,2024-01-11T20:08:45,Отдельно горжусь самостоятельно найденной иллю...,
6166,2024-01-11T20:08:45,,
6167,2024-01-11T20:33:29,Давно что-то мы с Юлией Леонидовной не беседов...,
6168,2024-01-12T15:36:44,(мрачное с утра)\n\nВ рейтинге самым популярны...,


In [ ]:
# checking if yebala is removed
tgposts.at[6167, 'text']

'Давно что-то мы с Юлией Леонидовной не беседовали о нашем, о политико-филологическом. С самого прошлого года и не беседовали. Вот как раз сегодня в 21.00 по Москве и 19.00 по Берлину поговорим о дискурсе деколонизации, а также DEI (diversity, equity, inclusion) повеcтке в целом, и не закат ли её мы наблюдаем (или не наблюдаем). \n\n \n\nhttps://www.youtube.com/live/iNG4zbE4WL0?si=uakOMhZDDpT37-aS'

In [ ]:
# preprocessing, step 1: getting rid of punctuation marks, links and cappital letters

def preprocess(tgdata):
  clean = re.sub(r'http\S{17,}', "", tgdata)
  clean = re.sub(r'[("){}<>=«»&ー;:,.?!_…@#$%¿¡]', "", clean)
  clean = re.sub(r'\s{2,}', " ", clean)
  tokenized = word_tokenize(clean)
  lowered = [i.lower() for i in tokenized]
  stopwords= nltk.corpus.stopwords.words('russian')
  wostop = [i for i in lowered if ( i not in stopwords )]
  morph = pymorphy2.MorphAnalyzer()
  lem_stem =[morph.parse(i)[0].normal_form for i in wostop]
  return ' '.join(lem_stem)

In [ ]:
# preprocessing
tgposts['preprocessed'] = tgposts['text'].apply(str)
tgposts['preprocessed'] = tgposts['text'].apply(preprocess)
tgposts.head()

,date,text,preprocessed
2,2018-12-20T08:30:15,Два условия обеспечат вам победу во взаимодейс...,условие обеспечить победа взаимодействие бюрок...
3,2018-12-21T05:45:19,"Павел Чиков, конечно, человек-зверь, прям Росо...",павел чик человек-зверь прям росомаха железный...
4,2018-12-22T10:00:15,Ссылка на Новую газету об этом деле с подробно...,ссылка новый газета дело подробность который р...
5,2018-12-23T15:01:59,https://www.youtube.com/watch?v=nLGozExwvio,
6,2018-12-24T09:00:26,​​Часто спрашивают о чем почитать про историю ...,​​часто спрашивать почитать история современны...


In [ ]:
# checking NaNs
tgposts.isna().sum()

date            0
text            0
preprocessed    0
dtype: int64

In [ ]:
# saving dataframe to csv file
tgposts.to_csv('/content/drive/MyDrive/Pet projects/Wordplays/Preprocessed texts.csv', index = False)

In [ ]:
# checking if the file is readable
tgposts = pd.read_csv('/content/drive/MyDrive/Pet projects/Wordplays/Preprocessed texts.csv')
tgposts.head()

,date,text,preprocessed
0,2018-12-20T08:30:15,Два условия обеспечат вам победу во взаимодейс...,условие обеспечить победа взаимодействие бюрок...
1,2018-12-21T05:45:19,"Павел Чиков, конечно, человек-зверь, прям Росо...",павел чик человек-зверь прям росомаха железный...
2,2018-12-22T10:00:15,Ссылка на Новую газету об этом деле с подробно...,ссылка новый газета дело подробность который р...
3,2018-12-23T15:01:59,https://www.youtube.com/watch?v=nLGozExwvio,NaN
4,2018-12-24T09:00:26,​​Часто спрашивают о чем почитать про историю ...,​​часто спрашивать почитать история современны...


In [ ]:
# checking NaNs
tgposts.isna().sum()

date               0
text            1058
preprocessed    1078
dtype: int64

In [ ]:
# dropping NaNs
tgposts.dropna(inplace = True)
tgposts.shape

(5073, 3)

In [ ]:
# saving dataframe to csv file
tgposts.to_csv('/content/drive/MyDrive/Pet projects/Wordplays/Ready posts.csv', index = False)

In [ ]:
# checking if the file is readable
tgposts = pd.read_csv('/content/drive/MyDrive/Pet projects/Wordplays/Ready posts.csv')
tgposts.head()

,date,text,preprocessed
0,2018-12-20T08:30:15,Два условия обеспечат вам победу во взаимодейс...,условие обеспечить победа взаимодействие бюрок...
1,2018-12-21T05:45:19,"Павел Чиков, конечно, человек-зверь, прям Росо...",павел чик человек-зверь прям росомаха железный...
2,2018-12-22T10:00:15,Ссылка на Новую газету об этом деле с подробно...,ссылка новый газета дело подробность который р...
3,2018-12-24T09:00:26,​​Часто спрашивают о чем почитать про историю ...,​​часто спрашивать почитать история современны...
4,2018-12-24T09:30:09,"​​Предыдущая книга для тех, кто может читать н...",​​предыдущий книга тот читать английский хорош...
